In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import *

In [5]:


def extract_sift(img):
    sift = cv2.SIFT_create(nfeatures=4000)
    kp, des = sift.detectAndCompute(img, None)
    return kp, des
    
def match_sift(des1, des2, ratio=0.75):
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    knn = flann.knnMatch(des1, des2, k=2)

    good = []
    for m, n in knn:
        if m.distance < ratio * n.distance:
            good.append(m)

    return knn, good

def ransac_affine(kp_d, kp_s, matches, thresh=6):
    if len(matches) < 12:
        return None, None, 0

    src = np.float32([kp_d[m.queryIdx].pt for m in matches])
    dst = np.float32([kp_s[m.trainIdx].pt for m in matches])

    A, mask = cv2.estimateAffinePartial2D(
        src, dst,
        method=cv2.RANSAC,
        ransacReprojThreshold=thresh,
        maxIters=5000,
        confidence=0.99
    )

    if mask is None:
        return None, None, 0

    return A, mask.ravel(), int(mask.sum())
def ransac_affine(kp_d, kp_s, matches, thresh=6):
    if len(matches) < 12:
        return None, None, 0

    src = np.float32([kp_d[m.queryIdx].pt for m in matches])
    dst = np.float32([kp_s[m.trainIdx].pt for m in matches])

    A, mask = cv2.estimateAffinePartial2D(
        src, dst,
        method=cv2.RANSAC,
        ransacReprojThreshold=thresh,
        maxIters=5000,
        confidence=0.99
    )

    if mask is None:
        return None, None, 0

    return A, mask.ravel(), int(mask.sum())
def affine_matching_with_visualization(drone_gray, sat_tile):
    """
    drone_gray : grayscale drone image
    sat_tile   : grayscale satellite tile
    """

    # --- Feature extraction ---
    kp_d, des_d = extract_sift(drone_gray)
    kp_s, des_s = extract_sift(sat_tile)

    if des_d is None or des_s is None:
        print("No descriptors found")
        return

    # --- Matching ---
    raw_knn, good_matches = match_sift(des_d, des_s)
    print(f"Raw knn matches: {len(raw_knn)}")
    print(f"Good matches after ratio test: {len(good_matches)}")

    # --- RAW MATCH VISUALIZATION ---
    raw_vis = cv2.drawMatches(
        drone_gray, kp_d,
        sat_tile, kp_s,
        good_matches[:100],
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    plt.figure(figsize=(14,6))
    plt.imshow(raw_vis, cmap="gray")
    plt.title("Raw SIFT Matches (Before Affine)")
    plt.axis("off")
    plt.show()

    # --- AFFINE RANSAC ---
    A, mask, inliers = ransac_affine(kp_d, kp_s, good_matches)
    print("Affine inliers:", inliers)

    if inliers < 12:
        print("❌ Tile rejected: insufficient affine inliers")
        return

    # --- Extract inliers ---
    inlier_matches = [
        m for m, keep in zip(good_matches, mask) if keep
    ]

    # --- AFFINE INLIER VISUALIZATION ---
    affine_vis = cv2.drawMatches(
        drone_gray, kp_d,
        sat_tile, kp_s,
        inlier_matches,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    plt.figure(figsize=(14,6))
    plt.imshow(affine_vis, cmap="gray")
    plt.title(f"Affine Inlier Matches (After RANSAC) – {inliers} inliers")
    plt.axis("off")
    plt.show()

    # --- WARP DRONE KEYPOINTS ---
    pts_drone = np.float32(
        [kp_d[m.queryIdx].pt for m in inlier_matches]
    ).reshape(-1,1,2)

    pts_warped = cv2.transform(pts_drone, A)

    sat_vis = cv2.cvtColor(sat_tile, cv2.COLOR_GRAY2BGR)

    for pt in pts_warped:
        x, y = int(pt[0][0]), int(pt[0][1])
        if 0 <= x < sat_vis.shape[1] and 0 <= y < sat_vis.shape[0]:
            cv2.circle(sat_vis, (x, y), 3, (0,0,255), -1)

    plt.figure(figsize=(6,6))
    plt.imshow(sat_vis)
    plt.title("Warped Drone Keypoints on Satellite Tile (Affine)")
    plt.axis("off")
    plt.show()

    # --- OPTIONAL: WARP ENTIRE DRONE IMAGE ---
    warped_drone = cv2.warpAffine(
        drone_gray,
        A,
        (sat_tile.shape[1], sat_tile.shape[0])
    )

    overlay = cv2.addWeighted(
        sat_tile, 0.6,
        warped_drone, 0.4,
        0
    )

    plt.figure(figsize=(6,6))
    plt.imshow(overlay, cmap="gray")
    plt.title("Warped Drone Image over Satellite Tile")
    plt.axis("off")
    plt.show()

    print("✅ Affine alignment successful")


In [6]:
# Load satellite map
sat_img, sat_transform, sat_crs = load_satellite_map("task_cv_model/map.tif")

# Load drone images
drone_imgs = load_drone_images("task_cv_model/train_data/drone_images/")

# Preprocess drone images
drone_imgs_proc = preprocess_drone_images(drone_imgs)

sat_gray = normalize_uint8(satellite_to_gray(sat_img))

name = list(drone_imgs.keys())[200]
drone_gray = drone_imgs_proc[name]

In [ ]:
sat_tiles = tile_image(sat_gray)


# for patch, offset in sat_tiles:

#     affine_matching_with_visualization(drone_gray, patch)
sat_features = []   
for t in sat_tiles:
    kp, des = extract_sift(t["tile"])
    if des is not None:
        sat_features.append({
            "kp": kp,
            "des": des,
            "offset": t["offset"],
            "tile": t["tile"]
        })
    sat_tile = t["tile"] #sat_features[0]
    # print('sample_tile:', sample_tile.shape)
    # break
    kp_s, des_s = extract_sift(sat_tile)
    kp_d, des_d = extract_sift(drone_imgs_proc[name])
    
    if des_d is None or des_s is None:
            print("No descriptors found")
            continue
    
    # --- Matching ---
    raw_knn, good_matches = match_sift(des_d, des_s)
    # print(f"Raw knn matches: {len(raw_knn)}")
    # print(f"Good matches after ratio test: {len(good_matches)}")
    
    # --- RAW MATCH VISUALIZATION ---
    raw_vis = cv2.drawMatches(
        drone_gray, kp_d,
        sat_tile, kp_s,
        good_matches[:100],
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    
        
    # --- AFFINE RANSAC ---
    A, mask, inliers = ransac_affine(kp_d, kp_s, good_matches)
    # print("Affine inliers:", inliers)
    
    if inliers <12:
        # print("❌ Tile rejected: insufficient affine inliers")
        # return
        continue
    print("Affine inliers:", inliers)
    
    # --- Extract inliers ---
    inlier_matches = [
        m for m, keep in zip(good_matches, mask) if keep
    ]
    
    # --- AFFINE INLIER VISUALIZATION ---
    affine_vis = cv2.drawMatches(
        drone_gray, kp_d,
        sat_tile, kp_s,
        inlier_matches,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    plt.figure(figsize=(14,6))
    plt.imshow(raw_vis, cmap="gray")
    plt.title("Raw SIFT Matches (Before Affine)")
    plt.axis("off")
    plt.show()
    
    plt.figure(figsize=(14,6))
    plt.imshow(affine_vis, cmap="gray")
    plt.title(f"Affine Inlier Matches (After RANSAC) – {inliers} inliers")
    plt.axis("off")
    plt.show()
    
    # --- WARP DRONE KEYPOINTS ---
    pts_drone = np.float32(
        [kp_d[m.queryIdx].pt for m in inlier_matches]
    ).reshape(-1,1,2)
    
    pts_warped = cv2.transform(pts_drone, A)
    
    sat_vis = cv2.cvtColor(sat_tile, cv2.COLOR_GRAY2BGR)
    
    for pt in pts_warped:
        x, y = int(pt[0][0]), int(pt[0][1])
        if 0 <= x < sat_vis.shape[1] and 0 <= y < sat_vis.shape[0]:
            cv2.circle(sat_vis, (x, y), 3, (0,0,255), -1)
    
    plt.figure(figsize=(6,6))
    plt.imshow(sat_vis)
    plt.title("Warped Drone Keypoints on Satellite Tile (Affine)")
    plt.axis("off")
    plt.show()
    
    # --- OPTIONAL: WARP ENTIRE DRONE IMAGE ---
    warped_drone = cv2.warpAffine(
        drone_gray,
        A,
        (sat_tile.shape[1], sat_tile.shape[0])
    )
    
    overlay = cv2.addWeighted(
        sat_tile, 0.6,
        warped_drone, 0.4,
        0
    )
    
    plt.figure(figsize=(6,6))
    plt.imshow(overlay, cmap="gray")
    plt.title("Warped Drone Image over Satellite Tile")
    plt.axis("off")
    plt.show()
    
    print("✅ Affine alignment successful")
